In [ ]:
#r "nuget: Microsoft.SemanticKernel.Agents.Abstractions, 1.14.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.14.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.OpenAI, 1.14.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.14.1-alpha"

Installed Packages Microsoft.SemanticKernel.Agents.Abstractions, 1.14.1-alpha Microsoft.SemanticKernel.Agents.Core, 1.14.1-alpha Microsoft.SemanticKernel.Agents.OpenAI, 1.14.1-alpha Microsoft.SemanticKernel.Connectors.OpenAI, 1.14.1

In [ ]:
using System.ComponentModel;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.OpenAI;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Agents.Chat;

In [ ]:
var OaiApiKey = "<your OpenAI KEY>";

In [ ]:
Kernel kernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
                modelId: "gpt-4o",
                apiKey: OaiApiKey)
            .Build();

In [ ]:
// Lets given the Persona to Agents

string progamManager = """
    You are a Program Manager responsible for gathering user requirements and developing a detailed project plan for app development. You analyze needs, define functional specifications, and deliver comprehensive documentation including scope, timelines, and cost estimates.. 
""";

string softwareEngineer = """
   You are a Software Engineer tasked with building a web app using HTML and JavaScript based on the requirements provided by the Program Manager.
""";

string projectManager = """
    You are a Manager responsible for reviewing the Software Engineer’s code to ensure it meets all client requirements. As the quality gatekeeper, you verify that the final product aligns with specifications and is ready for release. If everything is in order, respond with 'approve'.
""";

In [ ]:
#pragma warning disable SKEXP0110, SKEXP0001 // Rethrow to preserve stack details

 ChatCompletionAgent progaramManagerAgent =
            new()
            {
                Instructions = progamManager,
                Name = "ProgaramManagerAgent",
                Kernel = kernel
            };

 ChatCompletionAgent softwareEngineerAgent =
            new()
            {
                Instructions = softwareEngineer,
                Name = "SoftwareEngineerAgent",
                Kernel = kernel
            };

 ChatCompletionAgent projectManagerAgent =
            new()
            {
                Instructions = projectManager,
                Name = "ProjectManagerAgent",
                Kernel = kernel
            };

In [ ]:
 #pragma warning disable SKEXP0110, SKEXP0001 // Rethrow to preserve stack details
    
    using System.Threading;
    
    private sealed class ApprovalTerminationStrategy : TerminationStrategy
    {
        // Terminate when the final message contains the term "approve"
        protected override Task<bool> ShouldAgentTerminateAsync(Agent agent, IReadOnlyList<ChatMessageContent> history, CancellationToken cancellationToken)
            => Task.FromResult(history[history.Count - 1].Content?.Contains("approve", StringComparison.OrdinalIgnoreCase) ?? false);
    }

In [ ]:
#pragma warning disable SKEXP0110, SKEXP0001 

        AgentGroupChat chat =
            new(progaramManagerAgent, softwareEngineerAgent, projectManagerAgent)
            {
                ExecutionSettings =
                    new()
                    {
                        TerminationStrategy =
                            new ApprovalTerminationStrategy()
                            {
                                Agents = [ProjectManagerAgent],
                                MaximumIterations = 6,
                            }
                    }
            };

In [ ]:
#pragma warning disable SKEXP0110, SKEXP0001 
        // Invoke chat and display messages.
        string input = """
        
        Create a basic calculator app with essential features. Keep it simple. Await manager’s approval before finalizing.
        """;

        chat.AddChatMessage(new ChatMessageContent(AuthorRole.User, input));
        Console.WriteLine($"# {AuthorRole.User}: '{input}'");

        await foreach (var content in chat.InvokeAsync())
        {
            Console.WriteLine($"# {content.Role} - {content.AuthorName ?? "*"}: '{content.Content}'");
        }